In [ ]:
pip install opendatasets

In [ ]:
!pip install transformers --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 103.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/nileshmalode1/samsum-dataset-text-summarization")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: SUMITHKUMAR0001
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/nileshmalode1/samsum-dataset-text-summarization


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

In [ ]:
train_data = pd.read_csv("/content/samsum-dataset-text-summarization/samsum-train.csv")

validation_data = pd.read_csv("/content/samsum-dataset-text-summarization/samsum-validation.csv")

In [ ]:
train_data = train_data.sample(n=6000,random_state=42).reset_index(drop=True)
validation_data = validation_data.sample(n=500, random_state=42).reset_index(drop=True)

In [ ]:
import re

def clean_text(text):

    if isinstance(text, str):
        text = re.sub(r'\r\n', '', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'<.*?>', '', text)
        text = text.strip().lower()
    else:
        text = str(text)
    return text

train_data['dialogue'] = train_data['dialogue'].apply(clean_text)
train_data['summary'] = train_data['summary'].apply(clean_text)

validation_data['dialogue'] = validation_data['dialogue'].apply(clean_text)
validation_data['summary'] = validation_data['summary'].apply(clean_text)

train_data.head()

,id,dialogue,summary
0,13811908,violet: hi! i came across this austin's articl...,violet sent claire austin's article.
1,13716431,pat: so does anyone know when the stream is go...,pat and lou are waiting for the stream but kev...
2,13810214,jane: jane: whaddya think? shona: this ur tind...,jane is updating her tinder profile tonight an...
3,13729823,"adam: do u have a map of paris?tom: yes, why? ...",tom has a map of paris.
4,13681400,"frank: hi, how's the family?mike: great! sam's...","mike is happy, because sam's moved out. mike a..."


In [ ]:
# Tokenization

tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples["dialogue"], padding = 'max_length', truncation=True, max_length=512)
    targets = tokenizer(examples["summary"], padding = 'max_length', truncation=True, max_length=150)
    inputs['labels'] = targets['input_ids']
    return inputs



train_dataset = train_data.apply(preprocess_function, axis=1)
validation_data = validation_data.apply(preprocess_function, axis=1)

In [ ]:
train_dataset[0]

{'input_ids': [25208, 10, 7102, 55, 3, 23, 764, 640, 48, 403, 17, 77, 31, 7, 1108, 11, 3, 23, 816, 24, 25, 429, 253, 34, 1477, 11275, 15, 17, 10, 3, 7997, 15, 10, 7102, 55, 3, 10, 61, 2049, 6, 68, 3, 23, 31, 162, 641, 608, 34, 5, 3, 10, 61, 7997, 15, 10, 68, 2049, 21, 1631, 81, 140, 3, 10, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = 10,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    warmup_steps = 500,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 50,
    save_steps = 500,
    eval_steps = 50,

    eval_strategy = "epoch",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = validation_data
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sumithkumar5701 (sumithkumar5701-servicio-de-rentas-internas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.517800,0.415119
2,0.423900,0.377030
3,0.385100,0.366470
4,0.385200,0.360245
5,0.387400,0.358162
6,0.353200,0.355517
7,0.360700,0.353928
8,0.358800,0.353244
9,0.364600,0.352374
10,0.343700,0.352006


TrainOutput(global_step=3750, training_loss=0.7794010462443034, metrics={'train_runtime': 2859.445, 'train_samples_per_second': 20.983, 'train_steps_per_second': 1.311, 'total_flos': 8120508088320000.0, 'train_loss': 0.7794010462443034, 'epoch': 10.0})

In [ ]:
model.save_pretrained("./saved_summary_model")
tokenizer.save_pretrained("./saved_summary_model")

('./saved_summary_model/tokenizer_config.json',
 './saved_summary_model/special_tokens_map.json',
 './saved_summary_model/spiece.model',
 './saved_summary_model/added_tokens.json')

In [ ]:
# Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./saved_summary_model")
tokenizer = T5Tokenizer.from_pretrained("./saved_summary_model")


In [ ]:
import torch

device = model.device

def summarize_dialogue(dialogue):
  dialogue = clean_text(dialogue)
  inputs = tokenizer.encode("summarize: " + dialogue, return_tensors="pt", max_length=512, truncation=True).to(device)
  outputs = model.generate(
      input_ids=inputs,
      max_length=150,
      num_beams=4,
      early_stopping=True
  )

  summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return summary

In [ ]:
sample_dialogue = """
Lenny: Babe, can you help me with something? Bob: Sure, what's up? Lenny: Which one should I pick? Bob: Send me photos Lenny: <file_photo> Lenny: <file_photo> Lenny: <file_photo> Bob: I like the first ones best Lenny: But I already have purple trousers. Does it make sense to have two pairs? Bob: I have four black pairs :D :D Lenny: yeah, but shouldn't I pick a different color? Bob: what matters is what you'll give you the most outfit options Lenny: So I guess I'll buy the first or the third pair then Bob: Pick the best quality then Lenny: ur right, thx Bob: no prob :
"""

# Use 'sample_dialogue' instead of 'sample.dialogue'
summary = summarize_dialogue(sample_dialogue)
print("summary: ", summary)

summary:  lenny has four black pairs. he will buy the first or the third pair.


In [ ]:
import shutil

model_dir = "/content/saved_summary_model"

output_zip_path = "/content/saved_summary_model.zip"

shutil.make_archive(base_name = "saved_summary_model", format = "zip", root_dir = model_dir)

'/content/saved_summary_model.zip'

In [ ]:
from IPython.display import FileLink # Corrected the typo from 'Ipythion' to 'IPython'

FileLink(output_zip_path)

/content/saved_summary_model.zip